In [12]:
# Importation des bibliothèques nécessaires
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [13]:
# 1. Chargement des données prétraitées
df = pd.read_csv('preprocessed_data.csv')

In [14]:
# 2. Sélection des caractéristiques (features) et de la variable cible (target)
X = df.drop(columns=['resale_price'])  # Caractéristiques
y = df['resale_price']  # Variable cible

In [15]:
# 3. Gestion des colonnes non numériques
# Convertir les colonnes de type date en format numérique (par exemple, années ou jours depuis une date de référence)
if 'lease_commence_date' in X.columns:
    X['lease_commence_date'] = pd.to_datetime(X['lease_commence_date'], errors='coerce')
    X['lease_commence_date'] = (X['lease_commence_date'] - X['lease_commence_date'].min()) / pd.Timedelta(days=1)  # Nombre de jours depuis la première date

# Encoder les variables catégorielles si nécessaire
for col in X.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))

In [18]:
# 4. Division des données en ensembles d'entraînement et de test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 5. Normalisation des données
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [17]:
# 6. Modèle de régression linéaire
from sklearn.linear_model import LinearRegression
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)

# Prédictions avec le modèle de régression linéaire
y_pred_lr = lr_model.predict(X_test_scaled)

# Évaluation du modèle de régression linéaire
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
mae_lr = mean_absolute_error(y_test, y_pred_lr)
mse_lr = mean_squared_error(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)

print("Régression Linéaire - Évaluation :")
print(f"MAE: {mae_lr}")
print(f"MSE: {mse_lr}")
print(f"R2: {r2_lr}")

Régression Linéaire - Évaluation :
MAE: 72928.70521472645
MSE: 10448395308.479979
R2: 0.6326645568368452


In [19]:
# 2. Entraînement du modèle XGBoost
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5)
model.fit(X_train_scaled, y_train)

# 3. Prédiction
y_pred = model.predict(X_test_scaled)

# 4. Évaluation
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# 5. Affichage des résultats
print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"R²: {r2}")

MAE: 40151.03920628834
MSE: 3579235005.6439905
R²: 0.8741644206439788
